In [4]:
import pandas as pd
import os
import numpy as np
from sklearn import preprocessing

import tensorflow as tf

import time
import sys
import sklearn
from sklearn.externals import joblib

# sys.path.append("C:/Users/Workstation/Desktop/tf_ml/Step3_ML1_Model")
# sys.path.append("/home/workstation/Desktop/tf_ml/Step3_ML1_Model")
# import MLPerformance_Class as mlp

from rnn_functions import *

print("ok")


ok


In [5]:
#Show Device
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

# os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)



In [6]:
#Parameters
NROWS = 9000

TARGET_TO_PREDICT = "EURUSD"

SEQ_LEN = 150
FUTURE_PERIOD_PREDICT = 5

NUMLAYER = 1
NEURONS = 30
DROPOUT = 0.3
LEARNING_RATE = 0.0005
BATCH_SIZE = 128
EPOCHS = 100
NAME = f"{TARGET_TO_PREDICT}-{SEQ_LEN}-{FUTURE_PERIOD_PREDICT}-{NUMLAYER}-{NEURONS}-{DROPOUT}-{LEARNING_RATE}-{BATCH_SIZE}-{EPOCHS}-{int(time.time())}"


In [7]:
#Load
df = pd.read_csv("./DATA/x_82_ETF_FOREX_MIN_RETONLY.csv", nrows = NROWS)
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date")


In [8]:
#Pre-process
df = create_target(df, TARGET_TO_PREDICT, 3)
df = classify_target(df, "target")

In [9]:
#Split
# time_points = ["20080510", "20080520", "20080610"]
# df_list = split_df_by_time(df, time_points = time_points)
# print(len(df_list))
# print([j.shape for j in df_list])

prop = [0.5, 0.7, 0.85]
df_list_2 = split_df_by_prop(df, prop = prop)
print(len(df_list_2))
print([j.shape for j in df_list_2])

4
[(4498, 49), (1799, 49), (1350, 49), (1350, 49)]


In [10]:
#Scaling
scaler = sklearn.preprocessing.StandardScaler()
train_x, train_y, scaler, x_columns = preprocess_returns_df(df=df_list_2[0], target_col = "target", scaler = scaler, SEQ_LEN = SEQ_LEN, fit = True, same_prop = True)
val_x, val_y, _, _ = preprocess_returns_df(df=df_list_2[1], target_col = "target", scaler = scaler, SEQ_LEN = SEQ_LEN, fit = False, same_prop = True)
joblib.dump(scaler, "./rnn_scaler.pkl")
joblib.dump(x_columns, "./x_columns.pkl")

['./x_columns.pkl']

In [11]:
val_x.shape

(1602, 150, 48)

In [9]:

#Training
data = (train_x, train_y, val_x, val_y)
model, history = cudnn_lstm(data, NUMLAYER, NEURONS, DROPOUT, LEARNING_RATE, BATCH_SIZE, EPOCHS, NAME, logs_folder = "logs_82", models_folder="models_82", device_name = "/gpu:1")


Train on 43506 samples, validate on 17610 samples
Epoch 1/100
43506/43506 [==============================] - 12s 285us/step - loss: 0.7857 - acc: 0.5027 - val_loss: 0.6964 - val_acc: 0.4998

Epoch 00001: val_acc improved from -inf to 0.49983, saving model to models/models_82/RNN_Final-001-0.500.model
Epoch 2/100
43506/43506 [==============================] - 11s 249us/step - loss: 0.7126 - acc: 0.5145 - val_loss: 0.7000 - val_acc: 0.5047

Epoch 00002: val_acc improved from 0.49983 to 0.50466, saving model to models/models_82/RNN_Final-002-0.505.model
Epoch 3/100
43506/43506 [==============================] - 11s 251us/step - loss: 0.7007 - acc: 0.5133 - val_loss: 0.6974 - val_acc: 0.5086

Epoch 00003: val_acc improved from 0.50466 to 0.50863, saving model to models/models_82/RNN_Final-003-0.509.model
Epoch 4/100
43506/43506 [==============================] - 11s 252us/step - loss: 0.6952 - acc: 0.5215 - val_loss: 0.6969 - val_acc: 0.5075

Epoch 00004: val_acc did not improve from 0.508

43506/43506 [==============================] - 11s 249us/step - loss: 0.6038 - acc: 0.6596 - val_loss: 0.7873 - val_acc: 0.5355

Epoch 00081: val_acc did not improve from 0.54361
Epoch 82/100
43506/43506 [==============================] - 11s 247us/step - loss: 0.6045 - acc: 0.6589 - val_loss: 0.7758 - val_acc: 0.5330

Epoch 00082: val_acc did not improve from 0.54361
Epoch 83/100
43506/43506 [==============================] - 11s 245us/step - loss: 0.6040 - acc: 0.6597 - val_loss: 0.7861 - val_acc: 0.5348

Epoch 00083: val_acc did not improve from 0.54361
Epoch 84/100
43506/43506 [==============================] - 11s 247us/step - loss: 0.6016 - acc: 0.6630 - val_loss: 0.7938 - val_acc: 0.5348

Epoch 00084: val_acc did not improve from 0.54361
Epoch 85/100
43506/43506 [==============================] - 11s 252us/step - loss: 0.6023 - acc: 0.6628 - val_loss: 0.8061 - val_acc: 0.5353

Epoch 00085: val_acc did not improve from 0.54361
Epoch 86/100
43506/43506 [===========================